## Middle Exam

#### 資料描述
| date | coupon id |  描述  |
|----:| --------:|-----:|
|  有   |       無      |   平常交易|
|  無   |       有      |   交易完成無使用coupon|
|  有   |       有      |   交易完成使用coupon|

In [104]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import warnings
data_path = 'data/MiddleExam/'
df = pd.read_csv(data_path + 'train_offline.csv')
df_test = pd.read_csv(data_path + 'test_offline.csv')
df = df[~df.Coupon_id.isna()]
df_test = df_test[~df_test.Coupon_id.isna()]
train_Y = df['Date'].apply(lambda x: 0 if pd.isnull(x) else 1)

df.head(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN
11,253750,8390,7531.0,20:5,0.0,20160327.0,NaN


In [105]:
## 特徵工程
df['User_id_hashes'] = df['User_id'].apply(lambda x: hash(x)%15)
merchant_count = df.groupby('Merchant_id')['Merchant_id'].agg({'Merchant_id_count':'size'}).reset_index()
df = pd.merge(df, merchant_count, on='Merchant_id', how='left')
discount_count = df.groupby('Discount_rate')['Discount_rate'].agg({'Discount_rate_count':'size'}).reset_index()
df = pd.merge(df, discount_count, on='Discount_rate', how='left')
distance_count = df.groupby('Distance')['Distance'].agg({'Distance_count':'size'}).reset_index()
df = pd.merge(df, distance_count, on='Distance', how='left')

# df = df.drop('User_id', 'Merchant_id', 'Coupon_id',axis=1)

df.head(10)


/Users/fan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/fan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """
/Users/fan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,User_id_hashes,Merchant_id_count,Discount_rate_count,Distance_count
0,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,8,32,14986,212424.0
1,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,8,32,14986,212424.0
2,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,14,122781,106875,212424.0
3,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,2,122781,106875,95013.0
4,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,8,122781,20085,61043.0
5,73611,2099,12034.0,100:10,NaN,20160207.0,NaN,6,16824,166693,NaN
6,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,1,33427,26864,149984.0
7,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,11,8321,106875,149984.0
8,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,12,122781,106875,61043.0
9,253750,8390,7531.0,20:5,0.0,20160327.0,NaN,10,690,47990,212424.0


In [96]:
merchant_count = df.groupby('Merchant_id')['Merchant_id'].agg({'Merchant_id_count':'size'}).reset_index()
merchant_count.head(1200)

/Users/fan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,Merchant_id,Merchant_id_count
0,8,2
1,13,3
2,14,10
3,15,79
4,17,26
5,18,6
6,19,6
7,20,31
8,21,7
9,23,5
